## 머신러닝 실습

### 텍스트 마이닝

#### 개요

##### 텍스트 마이닝
- 비정형 텍스트데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고, 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 이용
- 프로세스 - 텍스트 전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
- 텍스트 전처리 - 토큰화, 불용어 제거, 표제어 추출, 형태소 분석

##### 특성 벡터화 / 추츨
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
-BoW, Word2ve 라이브러리(모듈) 존재

##### LDA
- 문서의 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLDAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_train.txt, ratings_test.txt 다운로드

- id(리뷰번호), document(리뷰), label(감성분류클래스 0은 부정, 1은 긍정 감성)

In [1]:
## 필요 라이브러리 사용 등록
import pandas as pd
import numpy as np
import re

In [2]:
# 모든 모듈/라이브러리는 __version__
np.__version__

'1.26.4'

In [3]:
## pandas 버전 상세정보
# pd.show_versions()

In [3]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [5]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [4]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
dfNsmcTrain.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [6]:
## 결측치 확인
dfNsmcTrain[dfNsmcTrain['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [7]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [8]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [9]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [10]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x:re.sub(r'[^가-힣|ㄱ-]+',' ',x))

In [11]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [12]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [13]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [14]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [15]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [16]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [17]:
## Regular Expression(정규식)으로 한글 이외의 문제는 다 공백으로 변경
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x:re.sub(r'[^가-힣|ㄱ-]+',' ',x))

In [18]:
dfNsmcTest

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [19]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] !=' ']

In [20]:
dfNsmcTrain

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1


In [21]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != ' ']

In [22]:
dfNsmcTest

,id,document,label
0,6270596,굳,1
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
5,7898805,음악이 주가 된 최고의 음악영화,1
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키지의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템등록정보 JAVA_HOME 설정
    - 프로그램 실행 : sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템 변수 > 새로 만들기 버튼 클릭
    - JAVA_HOME 과 경로 추가 등록
- JPype1 : 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [32]:
# Jpype1 설치
!pip install JPype1

   ---------------------------------------- 0.0/352.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/352.2 kB ? eta -:--:--
   ------------------- -------------------- 174.1/352.2 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 352.2/352.2 kB 3.1 MB/s eta 0:00:00


In [33]:
# koNLPy 설치
!pip install koNLPy 

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB 653.6 kB/s eta 0:00:30
   ---------------------------------------- 0.2/19.4 MB 2.0 MB/s eta 0:00:10
    --------------------------------------- 0.4/19.4 MB 3.1 MB/s eta 0:00:07
   - -------------------------------------- 0.6/19.4 MB 3.1 MB/s eta 0:00:07
   - -------------------------------------- 0.8/19.4 MB 3.3 MB/s eta 0:00:06
   - -------------------------------------- 1.0/19.4 MB 3.4 MB/s eta 0:00:06
   -- ------------------------------------- 1.2/19.4 MB 3.8 MB/s eta 0:00:05
   -- ------------------------------------- 1.4/19.4 MB 3.8 MB/s eta 0:00:05
   --- ------------------------------------ 1.7/19.4 MB 4.0 MB/s eta 0:00:05
   ---- ----------------------------------- 2.0/19.4 MB 4.2 MB/s eta 0:00:05
   ----- ---------------------------------- 2.5/19.4 MB 4.9 MB/s eta 0:00:04
   ----- --

In [23]:
import konlpy

In [24]:
konlpy.__version__

'0.6.0'

In [25]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
okt = Okt()

In [27]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [28]:
# 벡터화 시켜주는 객체
tfidf = TfidfVectorizer(tokenizer= oktTokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)

In [29]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
print(nsmc_train_tfidf)

  (0, 99083)	0.4538419136987346
  (0, 98558)	0.5247137293198487
  (0, 98114)	0.1845131235187482
  (0, 56963)	0.21259654811097825
  (0, 37450)	0.3275415869513754
  (0, 23765)	0.4840559509707182
  (0, 23738)	0.3128369859229991
  (1, 113534)	0.2112011196987848
  (1, 105850)	0.2706343581547997
  (1, 105835)	0.18835202148080835
  (1, 101049)	0.2769867495956953
  (1, 101037)	0.19619237668389927
  (1, 96060)	0.1650101022631291
  (1, 93754)	0.211675592928813
  (1, 69903)	0.2758246398817309
  (1, 69896)	0.22637757282967996
  (1, 68973)	0.32164169154818506
  (1, 68087)	0.06455439367805471
  (1, 67232)	0.32164169154818506
  (1, 67103)	0.12049482241316006
  (1, 59674)	0.3279940829890806
  (1, 43019)	0.3279940829890806
  (1, 42734)	0.1283430041888947
  (1, 1891)	0.2736337333728211
  (2, 108786)	0.2596474453876755
  :	:
  (148432, 72373)	0.33492725624458786
  (148432, 68796)	0.17275479979307826
  (148432, 68087)	0.06591895120816439
  (148432, 41285)	0.3231405928432333
  (148432, 41284)	0.25873499761

In [31]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)

##### 감성분류 모델 구축

In [33]:
## 로지스틱 회귀를 위한 라이브러리 등록
from sklearn.linear_model import LogisticRegression

In [34]:
model = LogisticRegression(random_state=0)

In [36]:
# 감성 이진분류 모델 훈련
model.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

LogisticRegression(random_state=0)

In [38]:
# 로지스틱회귀에 사용될 하이퍼파라미터 리스트
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [39]:
## 최적 예측모델을 찾기위한 작업
# 추가 라이브러리 사용등록
from sklearn.model_selection import GridSearchCV

In [40]:
## 하이퍼파라미터 중 C(현재 1.0) 값을 변경하면서 최적예측 모델 찾기
params = {'C': [3.0, 3.5, 4.0, 4.5, 5.0]}
testModel = GridSearchCV(model, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [41]:
testModel.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\DEV\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\DEV\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optim

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [3.0, 3.5, 4.0, 4.5, 5.0]}, scoring='accuracy',
             verbose=1)

In [45]:
testModel.best_estimator_

LogisticRegression(C=3.5, random_state=0)

In [46]:
# C=3.5로 해서 bestModel을 생성
bestModel = testModel.best_estimator_

##### 검증(평가)용 데이터 벡터화


In [47]:
nsmc_test_tfidf = tfidf.transform(dfNsmcTest['document'])

In [50]:
# 예측결과
y_predict = bestModel.predict(nsmc_test_tfidf)

In [53]:
# 감정분류 예측결과, 실제 라벨 수 동일
len(y_predict), len(dfNsmcTest['label'])

(49443, 49443)

In [48]:
# 평가지표로 정확도 확인
from sklearn.metrics import accuracy_score

In [54]:
# bestModel로 예측결과
accuracy_score(dfNsmcTest['label'], y_predict)

0.8624072163905911

##### 베스트모델로 감성예측

In [95]:
sentence = input('리뷰 입력 >>')

In [81]:
sentence

'비도 오고 우울해'

In [76]:
# # # 입력문장에서 한글만 추출 / 쓰지마세요
# st = re.compile(r'[가-힣]').findall(sentence)
# st

['웃', '자', '오', '늘', '은', '좋', '은', '날', '이', '될', '것', '같', '은', '예', '감']

In [96]:
st = re.sub(r'[^가-힣|ㄱ-]+', ' ', sentence)
st

'영화가 관람객들에게 보여줄려는게 뭘까'

In [98]:
## 리스트로 형변환
lstSt = [st]
lstSt

['영화가 관람객들에게 보여줄려는게 뭘까']

In [99]:
# 입력 테스트의 벡터화
st_tfidf = tfidf.transform(lstSt)
print(st_tfidf)

  (0, 68091)	0.19814907656386452
  (0, 68087)	0.09167741992275864
  (0, 65101)	0.22485992226969279
  (0, 39151)	0.34369155566377096
  (0, 39146)	0.26087425509823137
  (0, 27621)	0.27942076634792173
  (0, 27379)	0.13060483857574617
  (0, 12570)	0.27043961622596957
  (0, 8218)	0.4330184933298628
  (0, 8217)	0.36619300876334016
  (0, 5687)	0.42490564338777953
  (0, 5583)	0.1792114949255765
  (0, 71)	0.11426229300631317


In [100]:
# 0은 부정적인 감정, 1은 긍정적인 감정
str_predict = bestModel.predict(st_tfidf)

In [101]:
str_predict

array([0], dtype=int64)

##### 결론
- 감정예측은 오류가 많을 수 있다
- 많은 훈련량이 필요하고 머신러닝을 딥러닝과 결하하여 정확도를 높여야 함